In [9]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)
import math

from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.fileName=pathParts[4]
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        #print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0].tolist()
        self.y=data[:, 1].tolist()
        self.t=data[:, 2].tolist()
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            self.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure) > 0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (len(self.pressure) > 0 and self.pressure[len(self.pressure)-1] == 0):
                self.x.pop(len(self.pressure)-1)
                self.y.pop(len(self.pressure)-1)
                self.t.pop(len(self.pressure)-1)
                self.pressure.pop(len(self.pressure)-1)
        
        self.dx=difference(self.x)
        self.dy=difference(self.y)
        
        self.calculatePathTangentAngle()
        self.calculatePVM()
        self.dpvm=difference(self.pvm)
        self.dpta=difference(self.pta)
        self.lcr=self.calculateLCR()
        self.dlcr=difference(self.lcr)
        
        self.calculateTotalAccelerationMagnitude()
        self.dtam=difference(self.tam)
        #self.pta=math.atan(self.dy/self.dx)
        #self.pvm=math.sqrt(self.dx**2+self.dy*2)
        #self.dx=self.x.diff()
        #self.lcr=math.log(self.pvm/self.pta,10)

        self.x=normalize(self.x)
        self.y=normalize(self.y)
    
        if self.inputDevice != "finger":
            self.dpressure=difference(self.pressure)
            self.dpressure=normalize(self.dpressure)    
            self.pressure=normalize(self.pressure)
        
        self.pvm=normalize(self.pvm)
        self.pta=normalize(self.pta)
        self.tam=normalize(self.tam)
        self.dpvm=normalize(self.dpvm)
        self.dpta=normalize(self.dpta)
        self.dlcr=normalize(self.dlcr)
        
        self.ddx=difference(self.dx)
        self.ddy=difference(self.dy)
        
        '''
        print('self.x') 
        print(self.x) 
        print(self.filePath) 
        print(self.dx)  
        print(self.dy)
        print(self.dpressure)
        print(self.pta)
        print(self.pvm)
        #print(self.lcr)
        print(self.tam)
        '''
    
    def calculatePathTangentAngle(self):
        self.pta=[]
        for i in range(0,len(self.dx)):
            if(self.dx[i]==0):
                self.pta.append(math.atan(self.dy[i]/(self.dx[i] + 0.0000000000000001)))
            else:
                self.pta.append(math.atan(self.dy[i]/self.dx[i]))
        
    def calculatePVM(self):
        self.pvm=[]
        for i in range(0,len(self.dx)):
            self.pvm.append(math.sqrt(self.dx[i]**2+self.dy[i]**2))
            
    def calculateLCR(self):
        lcr=[]
        for i in range(0,len(self.dpta)):
            value=0
            if(self.dpta[i]==0):
                value=self.pvm[i]/(self.dpta[i]+0.0000000000000001)
            else:
                value=self.pvm[i]/self.dpta[i]
            value=abs(value)
            if(value<=0):
                '''
                print("value<=0 at index:")
                print(i)
                print('value:')
                print(value)
                print('self.dx')
                print(self.dx)
                print('self.dy') 
                print(self.dy)
                print('self.pta') 
                print(self.pta)
                print('self.dpta') 
                print(self.dpta)
                print('self.pvm')
                print(self.pvm)
                '''
                lcr.append(math.log(value+0.0000000000000001,10))
            else:
                lcr.append(math.log(value,10))
                
        return lcr
    
    def calculateTotalAccelerationMagnitude(self):
        self.tam=[]
        for i in range(0,len(self.dpvm)):
            self.pta.append(math.sqrt(self.dpvm[i]**2+(self.pvm[i]**2)*(self.pta[i]**2)))
            
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        '''
        print(split)
        print(inputDevice)
        print(signerId)
        '''
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

class Comparison:
    def __init__(self, file1, file2, label, filePath):
        self.file1=file1
        self.file2=file2
        self.label=label
        self.filePath=filePath   
        
def readFilesInDir(dirPath, allFiles):
    print('asdasd')
    signatureFiles = os.listdir(dirPath)
    i=0
    for file in signatureFiles:
        print(i)
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)
        if i%100==0:
            print(i)
        i+=1
        
def readComparisonFile(filePath, comparisons):
    dataframe=pd.read_csv(filePath,sep=' ',header=None)
    array=dataframe.to_numpy()
    #print(array)
    #print(len(array[:, 0]))
    for i in range(0,len(array[:, 0])-1):
        comparison=Comparison(array[i,0],array[i,1],array[i,2],filePath)
        comparisons.append(comparison)
        
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
    return diff

def Average(lst):
    return sum(lst) / len(lst)

def normalizeX(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.x
    
    #print('xAll')
    #print(xAll)
    #average=mean(xAll)
    z = stats.zscore(all)
    #print(average)
    print(z)
    allIndex=0
    
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized x:')
    for item in allFiles:
        print(item.x)
    '''
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=(item.x[i]-average)/z
    '''
def normalizeY(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.y
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.y),1):
            item.y[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized y:')
    for item in allFiles:
        print(item.y)
            
def normalizePressure(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.pressure
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized pressure:')
    for item in allFiles:
        print(item.filePath)
        print(item.pressure)

def normalize(data):
    normalized=[]
    z = stats.zscore(data)
    #print(z)

    for i in range(0,len(data),1):
        normalized.append(z[i])
            
    return normalized
'''
for item in normalized:
        print(item)
        
    return normalized
'''
    
        
def normalizeY2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.y)
        print(z)

        for i in range(0,len(item.y),1):
            item.y[i]=z[i]

    print('Normalized y:')
    for item in allFiles:
        print(item.y)
        
def normalizePressure2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.pressure)
        print('item.pressure:')
        print(item.pressure)
        print('z:')
        print(z)

        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[i]

    print('Normalized pressure:')
    for item in allFiles:
        print("item.filePath")
        print(item.pressure)

def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

def dtwAllX(comparisons,allFiles):
    dtwOutputs=[]
    comparisonCount=len(comparisons)
    for i in range(0,comparisonCount):
        print('findok')
        signature1=findByPath(allFiles,comparisons[i].file1)
        signature2=findByPath(allFiles,comparisons[i].file2)
        print('findok')
        #signature1=next((x for x in allFiles if x.fileName == comparisons[i].file1), None)
        #signature2=next((x for x in allFiles if x.fileName == comparisons[i].file2), None)
        if signature1 is None or signature2 is None:
            print('valamelyik signature nem található')
            print('i:')
            print(comparisons[i].file1)
            print(comparisons[i].file2)
            for file in allFiles:
                print(file.fileName)
        print('dtw')
        #dtwOutputs.append(fastdtw(signature1.x, signature2.x, dist=euclidean))
        comparisons[i].dtwx=fastdtw(signature1.x, signature2.x, dist=euclidean)
        comparisons[i].dtwy=fastdtw(signature1.y, signature2.y, dist=euclidean)
        if(signature1.inputDevice=='stylus'):
            comparisons[i].dtwpressure=fastdtw(signature1.pressure, signature2.pressure, dist=euclidean)
            comparisons[i].dtwdpressure=fastdtw(signature1.dpressure, signature2.dpressure, dist=euclidean)
        comparisons[i].dtwdx=fastdtw(signature1.dx, signature2.dx, dist=euclidean)
        comparisons[i].dtwdy=fastdtw(signature1.dy, signature2.dy, dist=euclidean)
        comparisons[i].dtwpta=fastdtw(signature1.pta, signature2.pta, dist=euclidean)
        comparisons[i].dtwpvm=fastdtw(signature1.pvm, signature2.pvm, dist=euclidean)
        
        comparisons[i].dtwdpvm=fastdtw(signature1.dpvm, signature2.dpvm, dist=euclidean)
        comparisons[i].dtwdpta=fastdtw(signature1.dpta, signature2.dpta, dist=euclidean)
        comparisons[i].dtwlcr=fastdtw(signature1.lcr, signature2.lcr, dist=euclidean)
        comparisons[i].dtwdlcr=fastdtw(signature1.dlcr, signature2.dlcr, dist=euclidean)
        comparisons[i].dtwtam=fastdtw(signature1.tam, signature2.tam, dist=euclidean)
        comparisons[i].dtwdtam=fastdtw(signature1.dtam, signature2.dtam, dist=euclidean)
        
        comparisons[i].dtwddx=fastdtw(signature1.ddx, signature2.ddx, dist=euclidean)
        comparisons[i].dtwddy=fastdtw(signature1.ddy, signature2.ddy, dist=euclidean) 
        
        print('dtw')
        print(comparisons[i].dtwddx)
        print(str(i)+'/'+str(comparisonCount))
        if i==10:
            return
        
    return dtwOutputs

def findByPath(allFiles,filePath):
    for file in allFiles:
        if file.fileName==filePath:
            return file
        
allFiles=[]
   
print('Reading signature files started')
#readFilesInDir('C:/DeepSignDB/Evaluation/stylus',allFiles)
readFilesInDir('C:/DeepSignDB/Evaluation/finger',allFiles)
#readFilesInDir('C:/DeepSignDB/Development/stylus',allFiles)
#readFilesInDir('C:/DeepSignDB/Development/finger',allFiles)
print('Reading signature files done')

comparisonFiles=[]
print('Reading comparison files started')
#readComparisonFile('C:/DeepSignDB/Comparison_Files/stylus/1vs1/random/Comp_DeepSignDB_random_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('C:/DeepSignDB/Comparison_Files/stylus/1vs1/skilled/Comp_DeepSignDB_skilled_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('C:/DeepSignDB/Comparison_Files/finger/1vs1/random/Comp_DeepSignDB_random_finger_1vs1.txt',comparisonFiles)
readComparisonFile('C:/DeepSignDB/Comparison_Files/finger/1vs1/skilled/Comp_DeepSignDB_skilled_finger_1vs1.txt',comparisonFiles)
print('Reading comparison files done')

'''
readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)

#readComparisonFile('D:/DeepSignSmall/Comparison_Files/stylus/1vs1/random/Comp_DeepSignDB_random_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('D:/DeepSignSmall/Comparison_Files/stylus/1vs1/skilled/Comp_DeepSignDB_skilled_stylus_1vs1.txt',comparisonFiles)
#readComparisonFile('D:/DeepSignSmall/Comparison_Files/finger/1vs1/random/Comp_DeepSignDB_random_finger_1vs1.txt',comparisonFiles)
readComparisonFile('D:/DeepSignSmall/Comparison_Files/finger/1vs1/skilled/Comp_DeepSignDB_skilled_finger_1vs1.txt',comparisonFiles)
'''
'''
print('Comparison files:')
for comparisonFile in comparisonFiles:
    print(comparisonFile.file1)
    print(comparisonFile.file2)
    print(comparisonFile.label)
    print(comparisonFile.filePath)
'''

'''
for file in allFiles:
    #print(file.data)
    #print(file.filePath)
    #print(file.db)
    #print(file.pressure)
'''

print('Allfiles:')
#print(allFiles)
print('Loading data finished')
dtwMatrix=dtwAllX(comparisonFiles,allFiles)
print(dtwMatrix)

#normalizeX2(allFiles)
#normalizeY2(allFiles)
#normalizePressure2(allFiles)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Reading signature files started
asdasd
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
24

1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
Reading signature files done
Reading comparison files started
Reading comparison files done
Allfiles:
Loading data finished
findok
findok
dtw
dtw
(1254.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 6), (12, 7), (13, 8), (14, 9), (15, 10), (15, 11), (16, 12), (17, 13), (18, 14), (18, 15), (19, 15), (20, 15), (21, 16), (22, 17), (23, 18), (24, 19), (24, 20), (25, 21), (26, 22), (27, 23), (28,

dtw
(1397.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 8), (12, 9), (13, 10), (14, 10), (15, 11), (16, 12), (17, 13), (18, 14), (19, 15), (20, 15), (20, 16), (21, 17), (22, 18), (23, 19), (24, 20), (24, 21), (25, 22), (25, 23), (25, 24), (26, 25), (26, 26), (26, 27), (27, 27), (28, 28), (29, 29), (30, 30), (30, 31), (30, 32), (30, 33), (30, 34), (30, 35), (30, 36), (30, 37), (30, 38), (31, 38), (32, 38), (33, 38), (34, 38), (35, 39), (36, 40), (37, 40), (38, 41), (39, 42), (39, 43), (39, 44), (40, 45), (41, 46), (42, 47), (43, 48), (44, 49), (45, 50), (45, 51), (46, 52), (47, 53), (47, 54), (48, 55), (49, 56), (49, 57), (49, 58), (49, 59), (49, 60), (49, 61), (49, 62), (49, 63), (49, 64), (49, 65), (50, 66), (50, 67), (50, 68), (50, 69), (50, 70), (50, 71), (50, 72), (50, 73), (50, 74), (50, 75), (50, 76), (50, 77), (50, 78), (50, 79), (50, 80), (50, 81), (50, 82), (50, 83), (50

dtw
(1649.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 11), (2, 11), (3, 11), (4, 11), (5, 11), (6, 11), (7, 11), (8, 11), (9, 11), (10, 11), (11, 12), (12, 12), (13, 13), (14, 14), (15, 15), (16, 15), (17, 16), (18, 17), (19, 18), (20, 18), (20, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (28, 29), (29, 29), (30, 29), (31, 29), (32, 29), (33, 29), (34, 29), (35, 30), (36, 30), (37, 30), (38, 30), (39, 31), (40, 32), (40, 33), (41, 34), (42, 35), (42, 36), (43, 37), (44, 38), (45, 39), (45, 40), (45, 41), (46, 42), (47, 43), (48, 44), (49, 45), (50, 45), (50, 46), (50, 47), (51, 48), (52, 49), (53, 49), (53, 50), (53, 51), (54, 51), (55, 51), (56, 51), (57, 52), (58, 53), (59, 54), (60, 55), (61, 56), (62, 56), (63, 56), (64, 56), (65, 56), (66, 56), (67, 56), (68, 56), (69, 56), (70, 57), (70, 58), (71, 59), (72, 60), (73, 60), (74, 61), (75, 61), (76, 62), (76, 63), (77, 64

dtw
(941.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (1, 12), (2, 12), (3, 12), (4, 12), (5, 12), (6, 12), (6, 13), (6, 14), (7, 14), (8, 14), (9, 14), (10, 14), (11, 15), (12, 15), (12, 16), (13, 17), (14, 18), (15, 19), (16, 20), (17, 21), (18, 22), (19, 23), (20, 24), (20, 25), (20, 26), (21, 26), (22, 27), (23, 27), (24, 28), (25, 28), (26, 29), (27, 29), (28, 30), (28, 31), (28, 32), (28, 33), (29, 34), (30, 35), (31, 35), (32, 35), (32, 36), (33, 36), (34, 36), (34, 37), (34, 38), (35, 39), (35, 40), (36, 41), (36, 42), (37, 42), (38, 43), (39, 43), (40, 44), (41, 44), (41, 45), (42, 46), (43, 47), (44, 48), (45, 49), (46, 49), (47, 50), (48, 51), (49, 52), (50, 52), (51, 52), (52, 52), (53, 53), (54, 53), (55, 53), (56, 53), (57, 54), (58, 55), (59, 56), (60, 57), (61, 58), (61, 59), (61, 60), (61, 61), (61, 62), (61, 63), (61, 64), (61, 65), (61, 66), (61, 67), (61, 68), (61, 69), (61, 70), (61, 71), (61, 72), (

dtw
(1093.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 6), (16, 7), (17, 8), (18, 9), (19, 9), (20, 10), (20, 11), (21, 12), (22, 13), (23, 14), (24, 15), (24, 16), (25, 17), (26, 18), (27, 19), (28, 20), (28, 21), (29, 22), (30, 23), (30, 24), (30, 25), (30, 26), (30, 27), (30, 28), (31, 28), (32, 28), (33, 28), (34, 28), (35, 29), (35, 30), (36, 31), (37, 31), (38, 32), (39, 33), (40, 34), (41, 34), (42, 34), (43, 34), (44, 35), (45, 36), (45, 37), (46, 38), (47, 39), (48, 40), (48, 41), (48, 42), (49, 43), (49, 44), (49, 45), (50, 46), (51, 47), (52, 48), (53, 49), (54, 49), (55, 49), (56, 49), (57, 50), (57, 51), (57, 52), (57, 53), (58, 54), (59, 55), (60, 56), (61, 57), (61, 58), (61, 59), (61, 60), (61, 61), (62, 61), (63, 61), (64, 61), (65, 61), (66, 61), (67, 61), (68, 62), (69, 62), (69, 63), (70, 64), (71, 65), (72, 66), (73, 67), (74, 68), (75, 

dtw
(1116.0, [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (10, 6), (11, 7), (12, 8), (12, 9), (12, 10), (13, 11), (14, 12), (15, 13), (16, 14), (17, 15), (18, 15), (19, 15), (19, 16), (20, 17), (20, 18), (20, 19), (21, 19), (22, 20), (23, 21), (24, 21), (25, 22), (26, 23), (27, 23), (28, 24), (28, 25), (28, 26), (28, 27), (29, 28), (30, 28), (31, 28), (32, 28), (33, 29), (34, 30), (35, 31), (36, 32), (37, 32), (38, 33), (39, 34), (40, 35), (41, 36), (42, 37), (43, 38), (44, 38), (45, 39), (46, 40), (47, 40), (48, 41), (49, 42), (49, 43), (49, 44), (49, 45), (49, 46), (50, 46), (51, 46), (52, 47), (53, 47), (53, 48), (53, 49), (54, 50), (55, 51), (56, 51), (57, 51), (58, 51), (59, 51), (60, 51), (61, 51), (62, 52), (62, 53), (63, 54), (64, 55), (65, 56), (66, 57), (66, 58), (67, 59), (68, 60), (68, 61), (69, 62), (69, 63), (69, 64), (70, 64), (71, 64), (72, 65), (73, 66), (73, 67), (74, 68), (75, 68), (76, 69), 